# Nettoyage des données V2

### Récupération des données

In [1]:
from scripts.importation import process_all_years_s3

In [3]:
import s3fs
import pandas as pd

In [4]:
# Connexion au bucket S3
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

ROOT_S3_PATH = "mvallat/diffusion/projet"

In [5]:
# Importation bucket S3
MY_BUCKET = "mvallat/diffusion"
FILE_PATH_S3 = f"{MY_BUCKET}/projet/data_brevets.parquet"

with fs.open(FILE_PATH_S3, 'rb') as file_in:
    data_brevets = pd.read_parquet(file_in)

data_brevets.head(10)

,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,...,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
0,3038352,A1,FR,PUBDEM,FR,3038352,20170106,2017-01,Brevet,ENSEMBLE POUR PANNEAU DE PORTE ET PANNEAU ASSOCIE,...,NA,NA,NA,2017,None,None,None,None,None,None
1,3038278,A1,FR,PUBDEM,FR,3038278,20170106,2017-01,Brevet,VEHICULE HYBRIDE COMPRENANT UNE SUPPRESSION DE...,...,NA,NA,NA,2017,None,None,None,None,None,None
2,3038409,A1,FR,PUBDEM,FR,3038409,20170106,2017-01,Brevet,PROCEDE DE CONCEPTION ASSISTEE PAR ORDINATEUR ...,...,NA,NA,NA,2017,None,None,None,None,None,None
3,3038425,A1,FR,PUBDEM,FR,3038425,20170106,2017-01,Brevet,DOCUMENT ELECTRONIQUE TEL QU'UNE CARTE A PUCE ...,...,NA,NA,NA,2017,None,None,None,None,None,None
4,3038316,A1,FR,PUBDEM,FR,3038316,20170106,2017-01,Brevet,PROCEDE DE SYNTHESE EN CONTINU D'UN ELASTOMERE...,...,NA,NA,NA,2017,None,None,None,None,None,None
5,3038399,A1,FR,PUBDEM,FR,3038399,20170106,2017-01,Brevet,PROCEDE ET DISPOSITIF DE COMMANDE D'UNE CONSIG...,...,NA,NA,NA,2017,None,None,None,None,None,None
6,3038381,A1,FR,PUBDEM,FR,3038381,20170106,2017-01,Brevet,DISPOSITIF CAPTEUR PREVU POUR ETRE EMBARQUE DE...,...,NA,NA,NA,2017,None,None,None,None,None,None
7,3038328,A1,FR,PUBDEM,FR,3038328,20170106,2017-01,Brevet,VEHICULE FERROVIAIRE POUR LE NETTOYAGE DES FIL...,...,NA,NA,NA,2017,None,None,None,None,None,None
8,3038251,A1,FR,PUBDEM,FR,3038251,20170106,2017-01,Brevet,PLANCHER PVC EN FIBRES COUPEES ET PROCEDE DE F...,...,NA,NA,NA,2017,None,None,None,None,None,None
9,3038374,A1,FR,PUBDEM,FR,3038374,20170106,2017-01,Brevet,INSTALLATION THERMIQUE DE SECHAGE DE MATIERE P...,...,NA,NA,NA,2017,None,None,None,None,None,None


### Observation des variables

#### **1) Type de brevet**

In [7]:
# Colonne "kind"
data_brevets["kind"].value_counts()

kind
A1    107749
A3      4441
NA       102
B1         3
Name: count, dtype: int64

Le document contient trois types de brevets :

- **A1** : demande de brevet  
- **A3** : demande de certificat d'utilité  
- **B1** : brevet délivré  


**Concernant les types B1 (n = 3)** : cette base recensant les demandes de brevet, elle n'est pas censée contenir un brevet B1.  
Probablement dû à une erreur de classification, nous pouvons donc retirer ces observations.

In [ ]:
# Suppression des B1
data_brevets_clean = data_brevets[data_brevets["kind"] != "B1"]

# Vérification qu'on a supprimé 3 observations
data_brevets_clean.shape[0]


112292

**Concernant les types A3 (n = 4 441)** : les certificats d'utilité sont des certificats délivrés par l'INPI qui protègent l'innovation, tout comme les brevets. En revanche la durée de protection est raccourcie (10 ans contre 20 ans) et la procédure d'examen de la demande est allégée. Pour simplifier l'analyse en la rendant plus homogène, nous ne gardons pas ces demandes A3 dans notre base.

In [10]:
# Suppression des A3
data_brevets_clean = data_brevets_clean[data_brevets_clean["kind"] != "A3"]

# Vérification : suppression de 4 441 observations (112,292 - 4,441 = 107,851)
data_brevets_clean.shape[0]

107851

**Concernant les NA (n = 102)** : plus d'observations sont nécessaires pour décider de la manière de gérer ces NA.


In [11]:
# Observation des NA
brevets_na = data_brevets[data_brevets["kind"] == "NA"]
brevets_na


,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,...,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
285,NA,NA,NA,NA,None,None,None,None,None,NA,...,NA,NA,NA,2017,None,None,None,None,None,None
606,NA,NA,NA,NA,None,None,None,None,None,NA,...,NA,NA,NA,2017,None,None,None,None,None,None
831,NA,NA,NA,NA,None,None,None,None,None,NA,...,NA,NA,NA,2017,None,None,None,None,None,None
1153,NA,NA,NA,NA,None,None,None,None,None,NA,...,NA,NA,NA,2017,None,None,None,None,None,None
1543,NA,NA,NA,NA,None,None,None,None,None,NA,...,NA,NA,NA,2017,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28236,NA,NA,NA,NA,None,None,None,None,None,NA,...,NA,NA,NA,2018,None,None,None,None,None,None
28556,NA,NA,NA,NA,None,None,None,None,None,NA,...,NA,NA,NA,2018,None,None,None,None,None,None
28760,NA,NA,NA,NA,None,None,None,None,None,NA,...,NA,NA,NA,2018,None,None,None,None,None,None
29096,NA,NA,NA,NA,None,None,None,None,None,NA,...,NA,NA,NA,2018,None,None,None,None,None,None


In [12]:
# Vérification du nombre d'informations réellement présentes
brevets_na["country"].nunique()
brevets_na["invention-title"].nunique()

2

On peut constater que seules deux observations contiennent des informations. Nous pouvons donc supprimer toutes les observations qui ne contiennent que des NA.

In [ ]:
# Suppression des observations 100% NA
data_brevets_clean = data_brevets_clean[data_brevets_clean["kind"] != "NA"]

#### **2) Numéro du document**

In [14]:
# Vérification que doc_number est unique

nb_number = data_brevets_clean["doc-number"].nunique()
print("Nombre de doc-number uniques :", nb_number)

nb_number2 = data_brevets_clean["publication_doc-number"].nunique()
print("Nombre de publication_doc-number uniques :", nb_number2)

nb_observations = data_brevets_clean.shape[0]
print("Nombre total d'observations :", nb_observations)


Nombre de doc-number uniques : 107748
Nombre de publication_doc-number uniques : 107748
Nombre total d'observations : 107749


Nous pouvons constater qu'il y a une observation qui n'a pas un numéro unique.

In [15]:
doublons = data_brevets_clean[data_brevets_clean.duplicated(subset=["doc-number"], keep=False)]
print(doublons)

      doc-number kind country  status publication_country  \
12476    3050837   A1      FR  PUBDEM                  FR   
58919    3050837   A1      FR     NEW                  FR   

      publication_doc-number publication_date publication_bopinum  \
12476                3050837         20171103             2017-44   
58919                3050837         20171103             2017-44   

      publication_nature                                    invention-title  \
12476             Brevet  DISPOSITIF D'INFORMATION INERTIELLE, SAGITTALE...   
58919  Demande française  DISPOSITIF D'INFORMATION INERTIELLE, SAGITTALE...   

       ... citation_3_country citation_3_doc-number citation_3_date  year  \
12476  ...                 NA                    NA              NA  2017   
58919  ...                 NA                    NA              NA  2020   

      application_country application_doc-number application_date  \
12476                None                   None             None   


#### **3) Statut**

La documentation de l'INPI nous apprend que la variable "status" est utilisée pour indiquer quel est l'évènement qui a entraîné la présence du brevet ou du CCP dans le fichier de la semaine. Les 

In [ ]:
# Colonne "status"
data_brevets_clean["status"].value_counts()

107749

In [21]:
data_brevets_clean.shape[0]

107749

In [22]:
data_brevets_clean["abstract"].nunique()


105885

### Tri des variables pertinentes

In [ ]:
data_brevets_light = data_brevets[['publication_date', 'invention-title', 'classification_1_text', 'abstract']]
data_brevets_clean.head(5)
